In [1]:
import pandas as pd

# Đọc dữ liệu từ file CSV
file_path = 'G:/BK/TTKH/matches.csv'  # Thay bằng đường dẫn tới file của bạn
data = pd.read_csv(file_path)

# Loại bỏ các cột không cần thiết
columns_to_drop = ["time", "referee", "match report", "notes", "attendance", "captain","date","day","formation","season"]
data_cleaned = data.drop(columns=columns_to_drop)

# Kiểm tra dữ liệu sau khi loại bỏ
print(data_cleaned.info())
print(data_cleaned.head())

# Lưu dữ liệu đã xử lý ra file mới (nếu cần)
data_cleaned.to_csv('cleaned_matches.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1389 entries, 0 to 1388
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1389 non-null   int64  
 1   comp        1389 non-null   object 
 2   round       1389 non-null   object 
 3   venue       1389 non-null   object 
 4   result      1389 non-null   object 
 5   gf          1389 non-null   float64
 6   ga          1389 non-null   float64
 7   opponent    1389 non-null   object 
 8   xg          1389 non-null   float64
 9   xga         1389 non-null   float64
 10  poss        1389 non-null   float64
 11  sh          1389 non-null   float64
 12  sot         1389 non-null   float64
 13  dist        1388 non-null   float64
 14  fk          1389 non-null   float64
 15  pk          1389 non-null   float64
 16  pkatt       1389 non-null   float64
 17  team        1389 non-null   object 
dtypes: float64(11), int64(1), object(6)
memory usage: 195.5+ KB
None
   

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import random
# 1. Xử lý giá trị thiếu
data_cleaned['dist'] = data_cleaned['dist'].fillna(data_cleaned['dist'].mean())
# 2. Mã hóa dữ liệu phân loại
categorical_cols = ['comp', 'round', 'venue', 'result', 'opponent', 'team']
label_encoders = {col: LabelEncoder() for col in categorical_cols}

for col in categorical_cols:
    data_cleaned[col] = label_encoders[col].fit_transform(data_cleaned[col])

# 3. Chọn các cột đầu vào và đầu ra
X = data_cleaned[['xg', 'xga', 'poss', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'opponent', 'team']]
y = data_cleaned[['gf', 'ga']]

# 4. Chuẩn hóa dữ liệu số
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. Chia dữ liệu thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Kiểm tra kết quả
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")


Shape of X_train: (1111, 11)
Shape of y_train: (1111, 2)
Shape of X_test: (278, 11)
Shape of y_test: (278, 2)


In [3]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
def U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6,
          wires):  
    """
    Mạch lượng tử U_9: một mạch với các phép Hadamard và điều khiển.
    
    Parameters:
        weights_2, weights_3, weights_4, weights_5, weights_6: Các tham số của mạch.
        wires: Các qubit sử dụng trong mạch.
    """
    qml.U3(*weights_0, wires=wires[0])
    qml.U3(*weights_1, wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(weights_2, wires=wires[0])
    qml.RZ(weights_3, wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(weights_4, wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(*weights_5, wires=wires[0])
    qml.U3(*weights_6, wires=wires[1])


In [4]:
# Thiết lập số lượng qubits và mạch lượng tử
n_qubits = 10
dev = qml.device("default.qubit", wires=n_qubits)

# Định nghĩa các tham số của mạch lượng tử
weight_shapes_QC = {
    "weights_0": 3,
    "weights_1": 3,
    "weights_2": 1,
    "weights_3": 1,
    "weights_4": 1,
    "weights_5": 3,
    "weights_6": 3,
}

# Mạch lượng tử
@qml.qnode(dev)
def qnode(inputs, weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6):
    """
    Định nghĩa mạch lượng tử cho mô hình HQCNN.
    """
    qml.AngleEmbedding(inputs, wires=range(n_qubits))  # Embedding dữ liệu vào mạch lượng tử
    
    # Sử dụng mạch U_9 trên các qubits
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[0, 1])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[2, 3])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[4, 5])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[6, 7])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[8, 9])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[1, 2])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[3, 4])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[5, 6])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[7, 8])
    U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[9, 0])
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

# Mô hình HQCNN
class HQCNN(nn.Module):
    def __init__(self, input_dim, qnode, weight_shapes, n_qubits):
        super(HQCNN, self).__init__()
        self.clayer_1 = nn.Linear(input_dim, n_qubits)  # Lớp chuyển đổi dữ liệu vào mạch lượng tử
        self.qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)  # Lớp lượng tử
        self.clayer_2 = nn.Linear(n_qubits, 2)  # Lớp đầu ra: [gf, ga]

    def forward(self, x):
        x = self.clayer_1(x)
        x = self.qlayer(x)
        x = self.clayer_2(x)
        return x


In [5]:
class HQCNN(nn.Module):
    def __init__(self, input_dim, qnode, weight_shapes, n_qubits):
        super(HQCNN, self).__init__()
        self.clayer_1 = nn.Linear(input_dim, n_qubits)  # Lớp chuyển đổi dữ liệu vào mạch lượng tử
        self.qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)  # Lớp lượng tử
        self.clayer_2 = nn.Linear(n_qubits, 2)  # Lớp đầu ra: [gf, ga]

    def forward(self, x):
        x = self.clayer_1(x)
        x = self.qlayer(x)
        x = self.clayer_2(x)
        return x
if not isinstance(X_train, torch.Tensor):
    X_train = torch.tensor(X_train, dtype=torch.float32)
if not isinstance(y_train, torch.Tensor):
    y_train = torch.tensor(y_train.values, dtype=torch.float32)
if not isinstance(X_test, torch.Tensor):
    X_test = torch.tensor(X_test, dtype=torch.float32)
if not isinstance(y_test, torch.Tensor):
    y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [6]:
# Khởi tạo mô hình
input_dim = X_train.shape[1]
model = HQCNN(input_dim, qnode, weight_shapes_QC, n_qubits)

# Cấu hình huấn luyện
criterion = nn.MSELoss()  # Hàm mất mát
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Bộ tối ưu hóa
epochs = 100

# Huấn luyện
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


Epoch 1/100, Loss: 3.60892915725708
Epoch 2/100, Loss: 3.477492094039917
Epoch 3/100, Loss: 3.34889817237854
Epoch 4/100, Loss: 3.2232487201690674
Epoch 5/100, Loss: 3.100114345550537
Epoch 6/100, Loss: 2.9791154861450195
Epoch 7/100, Loss: 2.8599586486816406
Epoch 8/100, Loss: 2.742432117462158
Epoch 9/100, Loss: 2.6264944076538086
Epoch 10/100, Loss: 2.5123324394226074
Epoch 11/100, Loss: 2.4004271030426025
Epoch 12/100, Loss: 2.2915360927581787
Epoch 13/100, Loss: 2.186723470687866
Epoch 14/100, Loss: 2.087294101715088
Epoch 15/100, Loss: 1.9946478605270386
Epoch 16/100, Loss: 1.9101072549819946
Epoch 17/100, Loss: 1.834714651107788
Epoch 18/100, Loss: 1.7690575122833252
Epoch 19/100, Loss: 1.7131173610687256
Epoch 20/100, Loss: 1.666180968284607
Epoch 21/100, Loss: 1.6269240379333496
Epoch 22/100, Loss: 1.5936553478240967
Epoch 23/100, Loss: 1.5646603107452393
Epoch 24/100, Loss: 1.5385075807571411
Epoch 25/100, Loss: 1.5142170190811157
Epoch 26/100, Loss: 1.4912731647491455
Epoch 

In [7]:
# Đánh giá trên tập test
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    mse = criterion(predictions, y_test)
    print(f"Mean Squared Error (MSE) on Test Data: {mse.item()}")


Mean Squared Error (MSE) on Test Data: 0.9093098640441895


In [8]:


def predict_score_qcnn(team_1, team_2, model, data_cleaned, scaler, label_encoders):
    """
    Dự đoán tỷ số giữa hai đội bằng mô hình Quantum CNN với dữ liệu lịch sử ngẫu nhiên.
    
    Parameters:
        team_1 (str): Tên đội chủ nhà.
        team_2 (str): Tên đội khách.
        model (HQCNN): Mô hình đã huấn luyện.
        data_cleaned (pd.DataFrame): Dữ liệu đã tiền xử lý.
        scaler (StandardScaler): Bộ chuẩn hóa dữ liệu.
        label_encoders (dict): Bộ mã hóa cho các cột phân loại.
    
    Returns:
        tuple: (gf, ga) - Tỷ số dự đoán giữa hai đội.
    """
    # Mã hóa tên đội
    if team_1 not in label_encoders['team'].classes_ or team_2 not in label_encoders['team'].classes_:
        raise ValueError("Tên đội không tồn tại trong dữ liệu!")
    
    team_1_encoded = label_encoders['team'].transform([team_1])[0]
    team_2_encoded = label_encoders['team'].transform([team_2])[0]

    # Lấy dữ liệu lịch sử của hai đội
    team_1_history = data_cleaned[data_cleaned['team'] == team_1_encoded]
    team_2_history = data_cleaned[data_cleaned['team'] == team_2_encoded]

    if team_1_history.empty or team_2_history.empty:
        raise ValueError("Không có đủ dữ liệu lịch sử cho một trong hai đội!")

    # Chọn một mẫu ngẫu nhiên từ lịch sử của mỗi đội
    team_1_sample = team_1_history.sample(n=1, random_state=random.randint(0, 1000)).iloc[0]
    team_2_sample = team_2_history.sample(n=1, random_state=random.randint(0, 1000)).iloc[0]

    # Tạo dữ liệu đầu vào từ mẫu ngẫu nhiên
    input_data = {
        'xg': team_1_sample['xg'],
        'xga': team_2_sample['xga'],
        'poss': (team_1_sample['poss'] + team_2_sample['poss']) / 2,
        'sh': (team_1_sample['sh'] + team_2_sample['sh']) / 2,
        'sot': (team_1_sample['sot'] + team_2_sample['sot']) / 2,
        'dist': (team_1_sample['dist'] + team_2_sample['dist']) / 2,
        'fk': (team_1_sample['fk'] + team_2_sample['fk']) / 2,
        'pk': (team_1_sample['pk'] + team_2_sample['pk']) / 2,
        'pkatt': (team_1_sample['pkatt'] + team_2_sample['pkatt']) / 2,
        'opponent': team_2_encoded,
        'team': team_1_encoded
    }

    # Chuẩn hóa đầu vào
    input_df = pd.DataFrame([input_data])
    input_scaled = scaler.transform(input_df)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32)

    # Dự đoán
    model.eval()  # Đặt mô hình ở chế độ đánh giá
    with torch.no_grad():
        prediction = model(input_tensor)
    
    predicted_gf, predicted_ga = prediction[0][0].item(), prediction[0][1].item()
    return round(predicted_gf), round(predicted_ga)


In [15]:
team_1 = "Tottenham"
team_2 = "Southampton"

import pandas as pd
import os

# Đường dẫn file
file_path = r'G:/BK/TTKH/predicted_scores.xlsx'

# Vòng lặp dự đoán
results = []
for i in range(20):
    try:
        # Dự đoán tỷ số
        predicted_score = predict_score_qcnn(team_1, team_2, model, data_cleaned, scaler, label_encoders)
        result = {
            "Team 1": team_1,
            "Team 2": team_2,
            "Score": f"{predicted_score[0]}-{predicted_score[1]}"
        }
        results.append(result)
    except ValueError as e:
        print(f"Lỗi dự đoán: {e}")

# Lưu kết quả vào file
if results:
    # Tạo DataFrame từ kết quả mới
    new_data = pd.DataFrame(results)
    
    # Nếu file đã tồn tại, đọc nội dung cũ và nối dữ liệu mới vào
    if os.path.exists(file_path):
        existing_data = pd.read_excel(file_path)
        all_data = pd.concat([existing_data, new_data], ignore_index=True)
    else:
        all_data = new_data
    
    # Ghi vào file
    all_data.to_excel(file_path, index=False, engine='openpyxl')
    print(f"Đã lưu {len(results)} kết quả mới vào {file_path}")
else:
    print("Không có kết quả để lưu.")


Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
Lỗi dự đoán: Tên đội không tồn tại trong dữ liệu!
